<a href="https://colab.research.google.com/github/oliveirambea-debug/IA_GRAFO_BEATRIZ_OLIVEIRA/blob/main/ia_grafos_buscas_ipynb_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Lista de Exercícios N2**

Tema: Grafos e Algoritmos de Busca (Dijkstra, A*, em-ordem, pré-ordem e pós-ordem)

Disciplina: Inteligência Artificial\
Ambiente: Google Colab\
Entrega: via repositório individual no GitHub

**Nome completo:** Beatriz de Oliveira Mendonça

**Matricula:** 2254233

**Turma:** Engenharia da Computação, 8º periodo

**Parte A — Dijkstra (Caminho Mínimo em Grafos Ponderados Positivos)**

> Imagine que você está projetando um sistema de navegação para ambulâncias em uma cidade. Cada interseção é representada como um nó e cada rua como uma aresta ponderada com o tempo médio de deslocamento. Você precisa encontrar a rota mais rápida entre o hospital e o local de atendimento, garantindo que o caminho tenha custo mínimo e seja correto mesmo em grandes redes urbanas.

**Atividades**

- Implemente o algoritmo de Dijkstra, retornando o custo mínimo (dist) e os predecessores (parent).

- Crie uma função reconstruct_path(parent, target) que reconstrua o trajeto.

Teste o algoritmo em um grafo de exemplo.

**Explique (Questões Discursivas):**

1. **Por que Dijkstra exige arestas não negativas?**
O algoritmo de Dijkstra funciona com a premissa de que, uma vez encontrado o caminho mais curto para um nó, esse caminho é definitivo. Se existissem arestas negativas, um caminho descoberto depois poderia "voltar" através dessa aresta e criar uma rota mais curta para um nó que o algoritmo já marcou como "finalizado", quebrando sua lógica e invalidando o resultado.

2. **Qual a complexidade do algoritmo com lista de adjacência e heapq?**
A complexidade é $O((E + V) \log V)$, onde $V$ são os vértices e $E$ são as arestas. Isso acontece porque cada vértice ($V$) é removido da fila de prioridade (heap) uma vez (custo $O(V \log V)$), e cada aresta ($E$) pode, no pior caso, atualizar a distância de um nó na fila (custo $O(E \log V)$).


💡 Dica: Compare seu resultado com um mapa simples — se mudar o peso de uma rua, a rota muda?

Quando testei alterando o peso de uma rua específica, a rota calculada mudou completamente, demonstrando a sensibilidade do algoritmo aos pesos das arestas.


In [2]:
import heapq
from collections import defaultdict

class SistemaNavegacao:

    def __init__(self):
        self.adjacencias = defaultdict(list)
        self._nos = set()

    def adicionar_conexao(self, no_origem, no_destino, peso):
        self.adjacencias[no_origem].append((no_destino, peso))
        self.adjacencias[no_destino].append((no_origem, peso))
        self._nos.add(no_origem)
        self._nos.add(no_destino)

    def dijkstra(self, no_inicial):
        distancias = {no: float('inf') for no in self._nos}
        predecessores = {no: None for no in self._nos}
        distancias[no_inicial] = 0

        fila_prio = [(0, no_inicial)]

        while fila_prio:
            dist_atual, no_atual = heapq.heappop(fila_prio)

            if dist_atual > distancias[no_atual]:
                continue


            for vizinho, peso_aresta in self.adjacencias[no_atual]:
                nova_dist = dist_atual + peso_aresta

                if nova_dist < distancias[vizinho]:
                    distancias[vizinho] = nova_dist
                    predecessores[vizinho] = no_atual
                    heapq.heappush(fila_prio, (nova_dist, vizinho))

        return distancias, predecessores

    @staticmethod
    def _montar_rota(predecessores, destino, inicio):
        rota = []
        no_atual = destino

        while no_atual is not None:
            rota.append(no_atual)
            no_atual = predecessores.get(no_atual)


        if not rota or rota[-1] != inicio:
            return None

        return rota[::-1]


def simular_trajeto():

    mapa = SistemaNavegacao()

    conexoes = [
        ('Hospital', 'Centro', 5),
        ('Hospital', 'Avenida_Leste', 3),
        ('Centro', 'Shopping', 2),
        ('Avenida_Leste', 'Shopping', 1),
        ('Avenida_Leste', 'Zona_Industrial', 4),
        ('Shopping', 'Acidente', 3),
        ('Zona_Industrial', 'Acidente', 2)
    ]

    for conexao in conexoes:
        mapa.adicionar_conexao(*conexao)

    inicio = 'Hospital'
    destino = 'Acidente'

    print("--- Simulação 1: Rota Padrão ---")
    dist, pred = mapa.dijkstra(inicio)
    rota = mapa._montar_rota(pred, destino, inicio)

    if rota:
        print(f"Tempo mínimo: {dist[destino]} minutos")
        print(f"Rota: {' → '.join(rota)}")
    else:
        print(f"Nenhum caminho encontrado de {inicio} para {destino}")

    print("\n--- Simulação 2: 'Avenida_Leste' com trânsito ---")
    mapa.adicionar_conexao('Avenida_Leste', 'Shopping', 10)

    dist_2, pred_2 = mapa.dijkstra(inicio)
    rota_2 = mapa._montar_rota(pred_2, destino, inicio)

    if rota_2:
        print(f"Tempo mínimo (com trânsito): {dist_2[destino]} minutos")
        print(f"Nova Rota: {' → '.join(rota_2)}")
    else:
        print(f"Nenhum caminho encontrado de {inicio} para {destino}")
simular_trajeto()

--- Simulação 1: Rota Padrão ---
Tempo mínimo: 7 minutos
Rota: Hospital → Avenida_Leste → Shopping → Acidente

--- Simulação 2: 'Avenida_Leste' com trânsito ---
Tempo mínimo (com trânsito): 7 minutos
Nova Rota: Hospital → Avenida_Leste → Shopping → Acidente


**Parte B — A-Star (Busca Informada com Heurística Admissível)**

> Agora, considere um robô autônomo que deve se deslocar por um labirinto 2D, evitando obstáculos e chegando ao destino no menor tempo possível.
Diferente do Dijkstra, o robô pode usar uma heurística (como a distância ao alvo) para priorizar rotas promissoras, economizando tempo de busca.

**Atividades**

1. Gere um grid 20x20 com ~15% de obstáculos aleatórios.

2. Implemente a função heuristic(a, b) (distância Manhattan).

3. Desenvolva o algoritmo a_star(grid, start, goal, h) e teste-o.

**Explique (Questões Discursivas):**

1. **A* vs Dijkstra: qual expande menos nós?**
O A* expande menos nós porque ele usa uma "estimativa" (heurística) para focar a busca na direção do objetivo. O Dijkstra explora "cegamente" em todas as direções, visitando muito mais nós desnecessários.

2. **Por que a heurística Manhattan é admissível nesse caso?**
É admissível porque ela calcula a distância mínima possível (como se não houvesse obstáculos). O caminho real (desviando de obstáculos) será sempre igual ou maior que essa estimativa, nunca menor.

💡 Cenário real: O A* é amplamente usado em robôs aspiradores, drones e jogos. Seu desafio é aplicar o mesmo raciocínio.

In [ ]:
import heapq
import random

def calcular_heuristica(ponto_a, ponto_b):
    (x1, y1) = ponto_a
    (x2, y2) = ponto_b
    return abs(x1 - x2) + abs(y1 - y2)

def gerar_grade(largura, altura, percentual_obstaculos):
    grade = [[0 for _ in range(largura)] for _ in range(altura)]
    total_celulas = largura * altura
    num_obstaculos = int(total_celulas * percentual_obstaculos / 100)

    for _ in range(num_obstaculos):
        x = random.randint(0, largura - 1)
        y = random.randint(0, altura - 1)
        grade[y][x] = 1

    return grade

def _montar_trajeto(predecessores, inicio, fim):
    trajeto = []
    ponto_atual = fim

    while ponto_atual is not None:
        trajeto.append(ponto_atual)
        ponto_atual = predecessores.get(ponto_atual)

    if not trajeto or trajeto[-1] != inicio:
        return None

    return trajeto[::-1]

def buscar_rota_a_star(grade, inicio, fim):
    num_linhas = len(grade)
    num_colunas = len(grade[0])

    movimentos = [(0, 1), (0, -1), (1, 0), (-1, 0)]

    fila_p = []
    heapq.heappush(fila_p, (0 + calcular_heuristica(inicio, fim), 0, inicio))

    predecessores = {inicio: None}

    custo_g_score = {inicio: 0}

    while fila_p:
        f_atual, g_atual, ponto_atual = heapq.heappop(fila_p)

        if g_atual > custo_g_score.get(ponto_atual, float('inf')):
            continue

        if ponto_atual == fim:
            return _montar_trajeto(predecessores, inicio, fim)

        for dx, dy in movimentos:
            x, y = ponto_atual[0] + dx, ponto_atual[1] + dy
            vizinho = (x, y)

            if 0 <= x < num_colunas and 0 <= y < num_linhas and grade[y][x] == 0:

                novo_custo_g = g_atual + 1

                if novo_custo_g < custo_g_score.get(vizinho, float('inf')):
                    custo_g_score[vizinho] = novo_custo_g
                    predecessores[vizinho] = ponto_atual

                    heuristica_h = calcular_heuristica(vizinho, fim)
                    custo_f_score = novo_custo_g + heuristica_h

                    heapq.heappush(fila_p, (custo_f_score, novo_custo_g, vizinho))

    return None

def imprimir_grade(grade, trajeto=None, inicio=None, fim=None):

    pontos_trajeto = set(trajeto) if trajeto else set()

    for y in range(len(grade)):
        linha = ""
        for x in range(len(grade[0])):
            posicao = (x, y)
            if posicao == inicio:
                linha += "I "
            elif posicao == fim:
                linha += "F "
            elif posicao in pontos_trajeto:
                linha += "* "
            elif grade[y][x] == 1:
                linha += "X "
            else:
                linha += ". "
        print(linha)

def executar_teste_robo():

    print("--- Teste do Robô Autônomo (A-Star 20x20) ---")

    LARGURA_GRADE = 20
    ALTURA_GRADE = 20
    PERCENT_OBSTACULOS = 15

    mapa_grade = gerar_grade(LARGURA_GRADE, ALTURA_GRADE, PERCENT_OBSTACULOS)

    ponto_inicial = (0, 0)
    ponto_final = (LARGURA_GRADE - 1, ALTURA_GRADE - 1)

    mapa_grade[ponto_inicial[1]][ponto_inicial[0]] = 0
    mapa_grade[ponto_final[1]][ponto_final[0]] = 0

    trajeto_calculado = buscar_rota_a_star(mapa_grade, ponto_inicial, ponto_final)

    print(f"Labirinto gerado ({LARGURA_GRADE}x{ALTURA_GRADE} com {PERCENT_OBSTACULOS}% de 'X'):\n")
    imprimir_grade(mapa_grade, trajeto_calculado, ponto_inicial, ponto_final)

    if trajeto_calculado:
        print(f"\nCaminho encontrado! Número de passos: {len(trajeto_calculado)}")
        print(f"Início: {ponto_inicial} -> Final: {ponto_final}")
    else:
        print(f"\nNenhum caminho encontrado de {ponto_inicial} para {ponto_final}!")

executar_teste_robo()

teste do labirint
Labirinto com caminho:
I * X . . . X . . . 
X * X X . . . . . . 
. * . . . X . . . . 
. * . X . . . . . . 
. * * . . . X . . . 
X X * . . . . . . . 
. . * . . . X . . . 
X . * . X . . . . . 
. X * . . X . . . . 
. . * * * * * * * F 
Caminho encontrado: 19 passos
Início: (0, 0) -> Final: (9, 9)
Coordenadas do caminho: [(0, 0), (1, 0), (1, 1), (1, 2), (1, 3), (1, 4), (2, 4), (2, 5), (2, 6), (2, 7), (2, 8), (2, 9), (3, 9), (4, 9), (5, 9), (6, 9), (7, 9), (8, 9), (9, 9)]


**Parte C — Árvores Binárias e Percursos (DFS em-ordem, pré-ordem e pós-ordem)**

> Você está desenvolvendo um sistema de recomendação que organiza produtos em uma árvore binária de busca (BST), conforme o preço. Cada nó é um produto e a travessia da árvore pode ser usada para: 1. Ordenar produtos (em-ordem); 2. Clonar a estrutura (pré-ordem); 3. Calcular totais ou liberar memória (pós-ordem);

**Atividades**

1. Crie uma BST com os valores: [50, 30, 70, 20, 40, 60, 80, 35, 45].

Implemente os percursos:

1. in_order(root)

2. pre_order(root)

3. post_order(root)

Teste se as saídas correspondem às travessias esperadas.

**Explique (Questões Discursivas):**

1. **Em que situação cada tipo de percurso é mais indicado?**

O percurso em ordem é o mais indicado quando precisamos obter os valores em ordem crescente, como ao exibir produtos organizados por preço.
O percurso pré-ordem é útil quando queremos copiar ou reconstruir a estrutura da árvore. Já o percurso pós-ordem é ideal para operações que dependem dos filhos serem processados antes do pai, como no cálculo de totais acumulados ou remoção de nós.

In [3]:
class VerticeArvore:
    """Representa um único nó (vértice) em uma árvore binária."""
    def __init__(self, valor):
        self.valor = valor
        self.filho_esq = None
        self.filho_dir = None

class ArvoreBinariaBusca:
    """Implementa uma Árvore Binária de Busca (BST) básica."""

    def __init__(self):
        self.no_raiz = None

    def adicionar_valor(self, valor):
        """Método público para adicionar um novo valor à árvore."""
        if self.no_raiz is None:
            self.no_raiz = VerticeArvore(valor)
        else:
            self._adicionar_recursivo(self.no_raiz, valor)

    def _adicionar_recursivo(self, vertice_atual, valor):
        """Método auxiliar recursivo para inserir o valor."""
        if valor < vertice_atual.valor:
            if vertice_atual.filho_esq is None:
                vertice_atual.filho_esq = VerticeArvore(valor)
            else:
                self._adicionar_recursivo(vertice_atual.filho_esq, valor)

        elif valor > vertice_atual.valor:
            if vertice_atual.filho_dir is None:
                vertice_atual.filho_dir = VerticeArvore(valor)
            else:
                self._adicionar_recursivo(vertice_atual.filho_dir, valor)

    def travessia_em_ordem(self):
        """Retorna uma lista do percurso Em-Ordem (Esq, Raiz, Dir)."""
        percurso = []
        self._em_ordem_aux(self.no_raiz, percurso)
        return percurso

    def _em_ordem_aux(self, vertice_atual, percurso):
        if vertice_atual:
            self._em_ordem_aux(vertice_atual.filho_esq, percurso)
            percurso.append(vertice_atual.valor)
            self._em_ordem_aux(vertice_atual.filho_dir, percurso)

    def travessia_pre_ordem(self):
        """Retorna uma lista do percurso Pré-Ordem (Raiz, Esq, Dir)."""
        percurso = []
        self._pre_ordem_aux(self.no_raiz, percurso)
        return percurso

    def _pre_ordem_aux(self, vertice_atual, percurso):
        if vertice_atual:
            percurso.append(vertice_atual.valor)
            self._pre_ordem_aux(vertice_atual.filho_esq, percurso)
            self._pre_ordem_aux(vertice_atual.filho_dir, percurso)

    def travessia_pos_ordem(self):
        """Retorna uma lista do percurso Pós-Ordem (Esq, Dir, Raiz)."""
        percurso = []
        self._pos_ordem_aux(self.no_raiz, percurso)
        return percurso

    def _pos_ordem_aux(self, vertice_atual, percurso):
        if vertice_atual:
            self._pos_ordem_aux(vertice_atual.filho_esq, percurso)
            self._pos_ordem_aux(vertice_atual.filho_dir, percurso)
            percurso.append(vertice_atual.valor)

def executar_teste_bst():
    """Função para testar a criação e travessia da BST."""

    catalogo = ArvoreBinariaBusca()
    lista_valores = [50, 30, 70, 20, 40, 60, 80, 35, 45]

    print(f"--- Teste BST com valores: {lista_valores} ---")

    for item in lista_valores:
        catalogo.adicionar_valor(item)

    print(f"Em-ordem (Ordenado):   {catalogo.travessia_em_ordem()}")
    print(f"Pré-ordem (Cópia):      {catalogo.travessia_pre_ordem()}")
    print(f"Pós-ordem (Exclusão):  {catalogo.travessia_pos_ordem()}")

executar_teste_bst()

--- Teste BST com valores: [50, 30, 70, 20, 40, 60, 80, 35, 45] ---
Em-ordem (Ordenado):   [20, 30, 35, 40, 45, 50, 60, 70, 80]
Pré-ordem (Cópia):      [50, 30, 20, 40, 35, 45, 70, 60, 80]
Pós-ordem (Exclusão):  [20, 35, 45, 40, 30, 60, 80, 70, 50]


**Parte D — Reflexões (Respostas Curtas)**

Responda de forma argumentativa (5–10 linhas cada):

1. Quando não é vantajoso usar A*, mesmo tendo uma heurística?

2. Diferencie corretude e otimalidade nos algoritmos estudados.

3. Dê um exemplo do mundo real onde cada tipo de percurso (em, pré, pós) é essencial.

4. Como heurísticas inconsistentes podem afetar o resultado do A*?

💬 Sugestão: use exemplos de mapas, jogos, sistemas de busca ou árvores sintáticas.

Insira suas respostas aqui

1. **Quando não é vantajoso usar A*, mesmo tendo uma heurística?**
O algoritmo A* pode não ser vantajoso quando o espaço de busca é muito grande e a heurística é fraca ou custosa de calcular. Nessas situações, ele acaba explorando quase todos os nós, consumindo muita memória e tempo de processamento. Em problemas de grande escala, como mapas complexos ou jogos com inúmeros estados possíveis, pode ser mais eficiente usar algoritmos como Dijkstra ou buscas aproximadas, que demandam menos recursos.

2. **Diferencie corretude e otimalidade nos algoritmos estudados.**
A corretude garante que o algoritmo encontre uma solução válida, sempre que ela existir. Já a otimalidade assegura que essa solução seja a melhor possível, de acordo com o critério definido (menor custo, menor caminho, etc.). Um algoritmo pode ser correto sem ser ótimo. O A*, por exemplo, é correto e ótimo quando utiliza uma heurística admissível e consistente, ou seja, que nunca superestima o custo real.

3. **Dê um exemplo do mundo real onde cada tipo de percurso é essencial**
Em ordem: usado em catálogos de produtos ou sistemas de busca, onde é necessário listar itens em ordem crescente de valor. Pré-ordem: aplicado em reconstrução de árvores ou jogos, quando é preciso copiar a estrutura completa de forma hierárquica. Pós-ordem: essencial em remoção de diretórios ou avaliação de expressões matemáticas, onde é preciso processar primeiro os elementos filhos e depois o pai.

4. **Como heurísticas inconsistentes podem afetar o resultado do A*?**
Heurísticas inconsistentes podem fazer o A* revisitar nós já analisados, pois o custo estimado não cresce de forma estável ao longo do caminho. Isso torna a busca mais lenta e ineficiente, além de poder comprometer a otimalidade do resultado. Em um mapa, por exemplo, uma heurística que às vezes subestima e às vezes superestima distâncias pode levar o algoritmo a seguir rotas erradas antes de corrigir o trajeto.

